<a href="https://colab.research.google.com/github/ValensioLeonard/Bangkit_Final_Capstone_Vivere/blob/main/Dental_Health_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dental Health Classifier, Bangkit Final Capstone

In [ ]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from shutil import copyfile
import re
from shutil import move


from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/Bangkit Final Project Files'

/content/gdrive/MyDrive/Bangkit Final Project Files


## Generate Dataset Folder

In [ ]:
# Define our directories and files
train_dir = 'Dataset/Training/'
validation_dir = 'Dataset/Validation/'

In [ ]:
# Use Temporary dir

# Populate Temporary Directory
def copy_file(source, destination):
  list_to_copy = os.listdir(source)
  for member in tqdm(list_to_copy):
    source_file = os.path.join(source, member)
    dest_file = os.path.join(destination, member)
    copyfile(source_file, dest_file)

copy_file(train_dir, 'Dataset/Dataset2/Training')
copy_file(validation_dir, 'Dataset/Dataset2/Validation')

100%|██████████| 113/113 [01:52<00:00,  1.00it/s]


In [ ]:
# Update Training and Validation Dir

train_dir = 'Dataset/Dataset2/Training'
validation_dir = 'Dataset/Dataset2/Validation'

## Move file to appropriate folder

In [ ]:
def move_to_class(dir):
  files = os.listdir(dir)
  for file in files:
    source_file = os.path.join(dir, file)
    try:
      if re.findall(r'^(.+)\.[0-9]+', file)[0] == 'no':
        dest_file = os.path.join(dir, 'Healthy' ,file)
        move(source_file, dest_file)
      else:
        dest_file = os.path.join(dir, 'Caries' ,file)
        move(source_file, dest_file)
    except:
      print('Skipped: ' + file)

move_to_class(train_dir)
move_to_class(validation_dir)

Skipped: Caries
Skipped: Healthy (1)
Skipped: Caries
Skipped: Healthy


## Convert file format to PNG

In [ ]:
from PIL import Image
import os

def convert_to_png(dir, verbose = False, remove_former = False):
  for file in os.listdir(dir):
    if file.endswith(".png") == False:
        im = Image.open(os.path.join(dir, file))
        name = re.findall(r'(.+)\.[\w+]+', file)[0] + '.png'        
        rgb_im = im.convert('RGB')
        rgb_im.save(os.path.join(dir, name))

        if remove_former:
          os.remove(os.path.join(dir, file))

        if verbose:
          print('converted ' + file + ' to ' + name)
        continue
    else:
        continue

In [ ]:
convert_to_png(os.path.join(train_dir, 'Healthy'), verbose = True, remove_former = True)
convert_to_png(os.path.join(train_dir, 'Caries'),  remove_former = True)
convert_to_png(os.path.join(validation_dir, 'Healthy'),  remove_former = True)
convert_to_png(os.path.join(validation_dir, 'Caries'),  remove_former = True)

## Initiate Image Generator

In [ ]:
train_heathy_dir = os.path.join(train_dir, 'Healthy')
train_caries_dir = os.path.join(train_dir, 'Caries')
validation_healthy_dir = os.path.join(validation_dir, 'Healthy')
validation_caries_dir = os.path.join(validation_dir, 'Caries')

train_heathy_fnames = os.listdir(train_heathy_dir)
train_caries_fnames = os.listdir(train_caries_dir)
validation_healthy_fnames = os.listdir(validation_healthy_dir)
validation_caries_fnames = os.listdir(validation_caries_dir)

print(len(train_heathy_fnames))
print(len(train_caries_fnames))
print(len(validation_healthy_fnames))
print(len(validation_caries_fnames))

56
57
24
24


In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255.,
                                   zoom_range = 0.2,
                                   shear_range = 0.2,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale = 1./255.)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    class_mode = 'binary',
                                                    batch_size = 10,
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                    class_mode = 'binary',
                                                    batch_size = 10,
                                                    target_size = (150, 150))

# Expected Output:
# Found 113 images belonging to 2 classes.
# Found 48 images belonging to 2 classes.

Found 113 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
